In [74]:
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [75]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [76]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [77]:
!wget -O longlat.csv -q https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=America/New_York&lang=en&use_labels_for_header=true&csv_separator=%3B

In [78]:
ls

col.csv                       spark-3.1.1-bin-hadoop2.7/
longlat.csv                   spark-3.1.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip  spark-3.1.1-bin-hadoop2.7.tgz.1
sample_data/


In [79]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [80]:
!pip install -q findspark

In [81]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [82]:
os.environ["PYSPARK_SUBMIT_ARGS"]="--master local[2] pyspark-shell"

In [83]:
import findspark
findspark.init()

In [84]:
import findspark
findspark.init()

In [85]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [86]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [87]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-05-07 14:55:39--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.225.89.236, 54.226.184.31, 34.239.208.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.225.89.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  6.16MB/s    in 2.1s    

2021-05-07 14:55:42 (6.16 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  t

In [88]:
!wget -q https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv ## Here we are downloading dataset for Collisions in New York City

In [89]:
mv rows.csv col.csv ## Linux cmd to rename dataset that we got in the above command.

In [90]:
ls

col.csv                         sample_data/
longlat.csv                     spark-3.1.1-bin-hadoop2.7/
ngrok-stable-linux-amd64.zip    spark-3.1.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip.1  spark-3.1.1-bin-hadoop2.7.tgz.1


In [91]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory","8G");
import numpy as np
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql.functions import round, col


In [92]:
spark =SparkSession(sc)

In [93]:

# Reading data from local repo. to pyspark.

col=spark.read.option("header",True).csv("./col.csv",inferSchema=True) # data set 1.
longlat=spark.read.option("delimiter",";").csv("./longlat.csv",inferSchema=True,header=True) #location. [Additional Dataset ]

In [94]:
col.count() - col.distinct().count() # checking for Duplicate rows in our dataset.

0

In [95]:
len(col.columns) # how many columns do I have.

29

In [96]:
col.printSchema() #Schema of my dataset. Issues with our columns types like NUMBER OF PERSONS INJURED: string and It should need to be Integer.

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: strin

In [97]:
from pyspark.sql.types import IntegerType

col=col.withColumn("NUMBER OF PERSONS INJURED", col["NUMBER OF PERSONS INJURED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF CYCLIST KILLED", col["NUMBER OF CYCLIST KILLED"].cast(IntegerType()))
col=col.withColumn("NUMBER OF MOTORIST INJURED", col["NUMBER OF MOTORIST INJURED"].cast(IntegerType()))

# Correcting the column types to Integer which were previously of String types.

In [98]:
col.printSchema() # checking if my schema is correct now ? Yes It is

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: st

In [99]:
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|04/14/2021|
|04/13/2021|
|04/15/2021|
|04/13/2021|
|04/12/2021|
|04/13/2021|
|04/13/2021|
|04/16/2021|
|04/11/2021|
|04/16/2021|
+----------+
only showing top 10 rows



In [100]:
#dropping columns with collision id = null

In [101]:
col.filter(col.COLLISION_ID.isNull()).drop()

DataFrame[CRASH DATE: string, CRASH TIME: string, BOROUGH: string, ZIP CODE: string, LATITUDE: double, LONGITUDE: double, LOCATION: string, ON STREET NAME: string, CROSS STREET NAME: string, OFF STREET NAME: string, NUMBER OF PERSONS INJURED: int, NUMBER OF PERSONS KILLED: int, NUMBER OF PEDESTRIANS INJURED: int, NUMBER OF PEDESTRIANS KILLED: int, NUMBER OF CYCLIST INJURED: int, NUMBER OF CYCLIST KILLED: int, NUMBER OF MOTORIST INJURED: int, NUMBER OF MOTORIST KILLED: int, CONTRIBUTING FACTOR VEHICLE 1: string, CONTRIBUTING FACTOR VEHICLE 2: string, CONTRIBUTING FACTOR VEHICLE 3: string, CONTRIBUTING FACTOR VEHICLE 4: string, CONTRIBUTING FACTOR VEHICLE 5: string, COLLISION_ID: int, VEHICLE TYPE CODE 1: string, VEHICLE TYPE CODE 2: string, VEHICLE TYPE CODE 3: string, VEHICLE TYPE CODE 4: string, VEHICLE TYPE CODE 5: string]

In [102]:
col.count()

1776922

In [103]:
col.filter(col["LATITUDE"].isNull()).count()

209909

In [104]:
col.filter(col["LONGITUDE"].isNull()).count()

209909

In [105]:
col.filter(col["LOCATION"].isNull()).count()

209909

In [106]:
col.filter(col["BOROUGH"].isNull()).count()

546563

In [107]:
col.filter(col["ZIP CODE"].isNull()).count()

546775

In [108]:
col.filter(col["ON STREET NAME"].isNull()).count()

358962

In [109]:
col.filter(col["OFF STREET NAME"].isNull()).count()

1510327

In [110]:
col.filter(col["CROSS STREET NAME"].isNull()).count()

627587

In [111]:
missingLocation=col.filter(col["BOROUGH"].isNull()).filter(col["ZIP CODE"].isNull())

In [112]:
cannotRetriveLocation = missingLocation.filter(col["LONGITUDE"].isNull())

In [113]:
col.count() ## removed the records for which we cannot retrive location at any cost.

1776922

In [114]:
#Infer the schema, and register the DataFrame as a table.
col.createOrReplaceTempView("col");
cannotRetriveLocation.createOrReplaceTempView("toDelete")

In [115]:
col=spark.sql("select * FROM col WHERE col.COLLISION_ID not in (select COLLISION_ID from toDelete where COLLISION_ID is not null)");


In [116]:
col.count()

1598841

In [117]:
col.filter(col["ZIP CODE"].isNull()).filter(col["BOROUGH"].isNotNull()).count()

212

In [118]:
BOROUGH=list(map(lambda x : x["BOROUGH"],col.select("BOROUGH").distinct().collect()))

In [119]:
BOROUGH

[None, 'QUEENS', 'BROOKLYN', 'BRONX', 'MANHATTAN', 'STATEN ISLAND']

In [120]:
col=col.fillna(value="Unknown",subset=["ZIP CODE"])

In [121]:
col=col.fillna(value="Unknown",subset=["BOROUGH","LOCATION","ON STREET NAME","OFF STREET NAME","CROSS STREET NAME"])

In [122]:
longlat.printSchema() # another data set

root
 |-- zip: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: integer (nullable = true)
 |-- dst: integer (nullable = true)
 |-- geopoint: string (nullable = true)



In [123]:
col.createOrReplaceTempView("col");
longlat.createOrReplaceTempView("longlat");

In [124]:
longlat.filter(longlat["zip"].isNull()).count()

0

In [125]:
longlat.select("zip").distinct().show()

+-----+
|  zip|
+-----+
|83861|
|67753|
|45011|
| 1580|
|80006|
|26623|
|15447|
|63106|
|76885|
|85321|
|43527|
|96044|
|40515|
| 2142|
|81410|
|53191|
|82730|
| 7833|
|57380|
|24171|
+-----+
only showing top 20 rows



In [126]:
longlat.filter(longlat["city"].isNull()).count()

0

In [127]:
longlat.filter(longlat["state"].isNull()).count()

0

In [128]:
longlat.filter(longlat["latitude"].isNull()).count()

0

In [129]:
longlat.filter(longlat["longitude"].isNull()).count()

0

In [130]:
longlat.filter(longlat["timezone"].isNull()).count()

0

In [131]:
longlat.filter(longlat["dst"].isNull()).count()

0

In [132]:
longlat.filter(longlat["geopoint"].isNull()).count()

0

In [133]:
missingZipCode=col.filter(col['ZIP CODE'].isNull());

In [134]:
missingZipCode.createOrReplaceTempView("missingZipCode");


In [135]:
col.select("NUMBER OF CYCLIST INJURED").distinct().show(); ## lets check.

+-------------------------+
|NUMBER OF CYCLIST INJURED|
+-------------------------+
|                        1|
|                        3|
|                        4|
|                        7|
|                        2|
|                        0|
+-------------------------+



NUMBER OF CYCLIST INJURED Is alredy cleaned.

In [136]:
col.select("NUMBER OF PERSONS INJURED").distinct().show();

+-------------------------+
|NUMBER OF PERSONS INJURED|
+-------------------------+
|                       31|
|                       27|
|                       12|
|                       22|
|                     null|
|                        1|
|                       13|
|                        6|
|                       16|
|                        3|
|                       20|
|                        5|
|                       19|
|                       15|
|                       43|
|                        9|
|                       17|
|                        4|
|                        8|
|                        7|
+-------------------------+
only showing top 20 rows



NUMBER OF PERSONS INJURED has Null value hence making it to Zero.

In [137]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS INJURED"]) # clean null values to zero.

In [138]:
#NUMBER OF PERSONS KILLED
print(col.select("NUMBER OF PERSONS KILLED").distinct().show());

+------------------------+
|NUMBER OF PERSONS KILLED|
+------------------------+
|                    null|
|                       1|
|                       3|
|                       5|
|                       4|
|                       8|
|                       2|
|                       0|
+------------------------+

None


NUMBER OF PERSONS KILLED has NULL values : replacing them with 0.


In [139]:
col=col.fillna(value=0,subset=["NUMBER OF PERSONS KILLED"])

In [140]:
print(col.select("NUMBER OF PEDESTRIANS INJURED").distinct().show());

+-----------------------------+
|NUMBER OF PEDESTRIANS INJURED|
+-----------------------------+
|                           27|
|                            1|
|                           13|
|                            6|
|                            3|
|                            5|
|                           15|
|                            9|
|                            4|
|                            7|
|                            2|
|                            0|
+-----------------------------+

None


NUMBER OF PEDESTRIANS INJURED has no null value

In [141]:
#NUMBER OF PEDESTRIANS KILLED
print(col.select("NUMBER OF PEDESTRIANS KILLED").distinct().show());

+----------------------------+
|NUMBER OF PEDESTRIANS KILLED|
+----------------------------+
|                           1|
|                           6|
|                           2|
|                           0|
+----------------------------+

None




```
NUMBER OF PEDESTRIANS KILLED has no null value
```



In [142]:
print(col.select("NUMBER OF CYCLIST INJURED").distinct().show());

+-------------------------+
|NUMBER OF CYCLIST INJURED|
+-------------------------+
|                        1|
|                        3|
|                        4|
|                        7|
|                        2|
|                        0|
+-------------------------+

None


NUMBER OF CYCLIST INJURED has no null value

In [143]:
print(col.select("NUMBER OF CYCLIST KILLED").distinct().show());

+------------------------+
|NUMBER OF CYCLIST KILLED|
+------------------------+
|                       1|
|                       2|
|                       0|
+------------------------+

None


NUMBER OF CYCLIST KILLED has no null value

In [144]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST INJURED").distinct().show(100));

+--------------------------+
|NUMBER OF MOTORIST INJURED|
+--------------------------+
|                        31|
|                        12|
|                        22|
|                         1|
|                        13|
|                         6|
|                        16|
|                         3|
|                        20|
|                         5|
|                        19|
|                        15|
|                        43|
|                         9|
|                        17|
|                         4|
|                         8|
|                         7|
|                        10|
|                        24|
|                        21|
|                        11|
|                        14|
|                         2|
|                        30|
|                         0|
|                        18|
+--------------------------+

None


NUMBER OF MOTORIST INJURED has no null value

In [145]:
#NUMBER OF MOTORIST INJURED
print(col.select("NUMBER OF MOTORIST KILLED").distinct().show(100));

+-------------------------+
|NUMBER OF MOTORIST KILLED|
+-------------------------+
|                        1|
|                        3|
|                        5|
|                        4|
|                        2|
|                        0|
+-------------------------+

None


NUMBER OF MOTORIST INJURED has no null value

In [146]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 1").distinct().show(70, truncate = False));

# has numerical values 1 and 80
# has null value.


+-----------------------------------------------------+
|CONTRIBUTING FACTOR VEHICLE 1                        |
+-----------------------------------------------------+
|Following Too Closely                                |
|Traffic Control Disregarded                          |
|Driverless/Runaway Vehicle                           |
|Accelerator Defective                                |
|Windshield Inadequate                                |
|Using On Board Navigation Device                     |
|Unsafe Speed                                         |
|Shoulders Defective/Improper                         |
|Tinted Windows                                       |
|Oversized Vehicle                                    |
|Passing or Lane Usage Improper                       |
|Lane Marking Improper/Inadequate                     |
|Other Lighting Defects                               |
|Aggressive Driving/Road Rage                         |
|null                                           

In [149]:
from pyspark.sql.functions import when

In [150]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 1', when(col["CONTRIBUTING FACTOR VEHICLE 1"] == "Drugs (illegal)", "Drugs (Illegal)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 1"]))

In [151]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 1', when(col["CONTRIBUTING FACTOR VEHICLE 1"] == "Cell Phone (hand-Held)", "Cell Phone (hand-held)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 1"]))

In [152]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 1', when(col["CONTRIBUTING FACTOR VEHICLE 1"] == "Illnes", "Illness").otherwise(col["CONTRIBUTING FACTOR VEHICLE 1"]))

In [153]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 1', when(col["CONTRIBUTING FACTOR VEHICLE 1"].isin("1","80"), "Others").otherwise(col["CONTRIBUTING FACTOR VEHICLE 1"]))

In [154]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 2").distinct().show(100));

# has numerical values 1 and 80.
# has null value.


+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 2|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|         Windshield Inadeq...|
|         Using On Board Na...|
|                 Unsafe Speed|
|         Shoulders Defecti...|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|         Other Lighting De...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|          Tow Hitch Defective|
|                      Texting|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         Cell Phone (hand-...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|       

In [155]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 2', when(col["CONTRIBUTING FACTOR VEHICLE 2"] == "Drugs (illegal)", "Drugs (Illegal)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 2"]))

In [156]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 2', when(col["CONTRIBUTING FACTOR VEHICLE 2"] == "Cell Phone (hand-Held)", "Cell Phone (hand-held)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 2"]))

In [157]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 2', when(col["CONTRIBUTING FACTOR VEHICLE 2"] == "Illnes", "Illness").otherwise(col["CONTRIBUTING FACTOR VEHICLE 2"]))

In [158]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 2', when(col["CONTRIBUTING FACTOR VEHICLE 2"].isin("1","80"), "Others").otherwise(col["CONTRIBUTING FACTOR VEHICLE 2"]))

In [159]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 3").distinct().show(100));

# has numerical values 1 and 80.
# has null value.

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 3|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Driverless/Runawa...|
|         Accelerator Defec...|
|                 Unsafe Speed|
|               Tinted Windows|
|            Oversized Vehicle|
|         Passing or Lane U...|
|         Lane Marking Impr...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|                       Illnes|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Other...|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Tire Failure/Inad...|
|         Outside Car Distr...|
|              Drugs (Illegal)|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|       

In [160]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 3', when(col["CONTRIBUTING FACTOR VEHICLE 3"] == "Drugs (illegal)", "Drugs (Illegal)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 3"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 3', when(col["CONTRIBUTING FACTOR VEHICLE 3"] == "Cell Phone (hand-Held)", "Cell Phone (hand-held)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 3"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 3', when(col["CONTRIBUTING FACTOR VEHICLE 3"] == "Illnes", "Illness").otherwise(col["CONTRIBUTING FACTOR VEHICLE 3"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 3', when(col["CONTRIBUTING FACTOR VEHICLE 3"].isin("1","80"), "Others").otherwise(col["CONTRIBUTING FACTOR VEHICLE 3"]))

In [161]:
#CONTRIBUTING FACTOR VEHICLE
print(col.select("CONTRIBUTING FACTOR VEHICLE 4").distinct().show(100));
# has null value.

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 4|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|         Accelerator Defec...|
|         Driverless/Runawa...|
|         Windshield Inadeq...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Prescription Medi...|
|         View Obstructed/L...|
|           Lost Consciousness|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Outside Car Distr...|
|              Drugs (Illegal)|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|         Unsafe Lane Changing|
|          Alcohol Involvement|
|                      Illness|
|       

In [162]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 4', when(col["CONTRIBUTING FACTOR VEHICLE 4"] == "Drugs (illegal)", "Drugs (Illegal)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 4"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 4', when(col["CONTRIBUTING FACTOR VEHICLE 4"] == "Cell Phone (hand-Held)", "Cell Phone (hand-held)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 4"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 4', when(col["CONTRIBUTING FACTOR VEHICLE 4"] == "Illnes", "Illness").otherwise(col["CONTRIBUTING FACTOR VEHICLE 4"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 4', when(col["CONTRIBUTING FACTOR VEHICLE 4"].isin("1","80"), "Others").otherwise(col["CONTRIBUTING FACTOR VEHICLE 4"]))

In [163]:
print(col.select("CONTRIBUTING FACTOR VEHICLE 5").distinct().show(30));
# has null values

+-----------------------------+
|CONTRIBUTING FACTOR VEHICLE 5|
+-----------------------------+
|         Following Too Clo...|
|         Traffic Control D...|
|                 Unsafe Speed|
|         Passing or Lane U...|
|                         null|
|         Aggressive Drivin...|
|              Other Vehicular|
|          Driver Inexperience|
|              Drugs (illegal)|
|                  Unspecified|
|           Pavement Defective|
|         Reaction to Uninv...|
|                  Fell Asleep|
|         Tire Failure/Inad...|
|         Outside Car Distr...|
|              Fatigued/Drowsy|
|         Driver Inattentio...|
|         Failure to Keep R...|
|           Obstruction/Debris|
|          Alcohol Involvement|
|                      Illness|
|         Traffic Control D...|
|             Brakes Defective|
|             Backing Unsafely|
|         Failure to Yield ...|
|                        Glare|
|             Steering Failure|
|          Passing Too Closely|
|       

In [250]:
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 5', when(col["CONTRIBUTING FACTOR VEHICLE 5"] == "Drugs (illegal)", "Drugs (Illegal)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 5"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 5', when(col["CONTRIBUTING FACTOR VEHICLE 5"] == "Cell Phone (hand-Held)", "Cell Phone (hand-held)").otherwise(col["CONTRIBUTING FACTOR VEHICLE 5"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 5', when(col["CONTRIBUTING FACTOR VEHICLE 5"] == "Illnes", "Illness").otherwise(col["CONTRIBUTING FACTOR VEHICLE 5"]))
col = col.withColumn('CONTRIBUTING FACTOR VEHICLE 5', when(col["CONTRIBUTING FACTOR VEHICLE 5"].isin("1","80"), "Others").otherwise(col["CONTRIBUTING FACTOR VEHICLE 5"]))

In [165]:
col.count()

1598841

In [166]:
## Setting all null values to Unknown 
col=col.fillna(value="Unknown",subset=["CONTRIBUTING FACTOR VEHICLE 1","CONTRIBUTING FACTOR VEHICLE 2","CONTRIBUTING FACTOR VEHICLE 3","CONTRIBUTING FACTOR VEHICLE 4","CONTRIBUTING FACTOR VEHICLE 5"])

In [167]:
## Checking what values in VEHICLE TYPE CODE need to be cleaned
print(col.select("VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5").distinct().show(30));


+--------------------+--------------------+--------------------+--------------------+--------------------+
| VEHICLE TYPE CODE 1| VEHICLE TYPE CODE 2| VEHICLE TYPE CODE 3| VEHICLE TYPE CODE 4| VEHICLE TYPE CODE 5|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Sedan|               MOPED|                null|                null|                null|
|          SKID LOADE|Station Wagon/Spo...|                null|                null|                null|
|           Box Truck|Station Wagon/Spo...|          Motorcycle|               Sedan|Station Wagon/Spo...|
|             TRAILER|       Pick-up Truck|                null|                null|                null|
|         Chassis Cab|Station Wagon/Spo...|               Sedan|                null|                null|
|                LIMO|          GAS SCOOTE|                null|                null|                null|
|           Lift Boom|           Box 

In [168]:
## Setting null values for vehicle type to unknown
col=col.fillna(value="Unknown",subset=["VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5"])

In [169]:
## Checking if the values were correctly changed
print(col.select("VEHICLE TYPE CODE 1","VEHICLE TYPE CODE 2","VEHICLE TYPE CODE 3","VEHICLE TYPE CODE 4","VEHICLE TYPE CODE 5").distinct().show(30, truncate = False));

+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+
|VEHICLE TYPE CODE 1                |VEHICLE TYPE CODE 2                |VEHICLE TYPE CODE 3                |VEHICLE TYPE CODE 4                |VEHICLE TYPE CODE 5                |
+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+-----------------------------------+
|van                                |Station Wagon/Sport Utility Vehicle|Unknown                            |Unknown                            |Unknown                            |
|Station Wagon/Sport Utility Vehicle|Station Wagon/Sport Utility Vehicle|Station Wagon/Sport Utility Vehicle|Dump                               |Unknown                            |
|Sedan                              |Multi-Wheeled Vehicle              |Unknown          

In [170]:
print(col.select("VEHICLE TYPE CODE 1").distinct().show(100, truncate = False));

+-----------------------------------+
|VEHICLE TYPE CODE 1                |
+-----------------------------------+
|truck trai                         |
|NIU                                |
|trail                              |
|BOX                                |
|USP M                              |
|HORSE CARR                         |
|CONVE                              |
|School Bus                         |
|Power Shov                         |
|TRK                                |
|FDNY LADDE                         |
|vespa                              |
|GOVER                              |
|BOAT                               |
|Station Wagon/Sport Utility Vehicle|
|TRAILER/FL                         |
|FREIG                              |
|Mta                                |
|COMMU                              |
|NYC M                              |
|yello                              |
|4dsd                               |
|Flat Bed                           |
|US POSTAL  

In [251]:
col = col.withColumn('VEHICLE TYPE CODE 1', when(col["VEHICLE TYPE CODE 1"] == "schoo", "School Bus").otherwise(col["VEHICLE TYPE CODE 1"]))
col = col.withColumn('VEHICLE TYPE CODE 1', when(col["VEHICLE TYPE CODE 1"] == "Ambul", "Ambulance").otherwise(col["VEHICLE TYPE CODE 1"]))
col = col.withColumn('VEHICLE TYPE CODE 1', when(col["VEHICLE TYPE CODE 1"] == "DELV", "Delivery vehicle").otherwise(col["VEHICLE TYPE CODE 1"]))
col = col.withColumn('VEHICLE TYPE CODE 1', when(col["VEHICLE TYPE CODE 1"] == "?omme", "Other").otherwise(col["VEHICLE TYPE CODE 1"]))

In [172]:
### Set all empty values in CROSS STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["CROSS STREET NAME"])

In [173]:
### Set all empty values in OFF STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["OFF STREET NAME"])

In [174]:
### Set all empty values in ON STREET NAME to No Value
col=col.fillna(value="NOT PROVIDED",subset=["ON STREET NAME"])

In [175]:
## Checking if results have been successfully changed
print(col.select("CROSS STREET NAME","OFF STREET NAME","ON STREET NAME").distinct().show(30));

+--------------------+--------------------+--------------------+
|   CROSS STREET NAME|     OFF STREET NAME|      ON STREET NAME|
+--------------------+--------------------+--------------------+
|     WEST 181 STREET|             Unknown|    AMSTERDAM AVENUE|
|             Unknown|1281      UNION A...|             Unknown|
|             Unknown|666       EAST 22...|             Unknown|
|             Unknown|             Unknown|       WEST 5 STREET|
|     CRESCENT STREET|             Unknown|      STANLEY AVENUE|
|             Unknown|64-06     FRESH P...|             Unknown|
|             Unknown|             Unknown|WEST BURNSIDE AVENUE|
|   WEST FORDHAM ROAD|             Unknown|MAJOR DEEGAN EXPR...|
|             Unknown|             Unknown|         ZEBRA PLACE|
|             Unknown|             Unknown|      EAST 22 STREET|
|             Unknown|1431      WEST 8 ...|             Unknown|
|           86 STREET|             Unknown|          102 AVENUE|
|     WEST 113 STREET|   

In [176]:
#Check CRASH DATE format
col.filter(col["CRASH DATE"].isNull()).show()

+----------+----------+-------+--------+--------+---------+--------+--------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|LATITUDE|LONGITUDE|LOCATION|ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING F

In [177]:
#Change CRASH DATE type from String to Date
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date

#col.withColumn("CRASH DATE", col['CRASH DATE'].cast(DateType())).show()

 
col= col.withColumn('CRASH DATE',to_date(col["CRASH DATE"], 'MM/dd/yyyy'))
col.printSchema()

root
 |-- CRASH DATE: date (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = false)
 |-- ZIP CODE: string (nullable = false)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LOCATION: string (nullable = false)
 |-- ON STREET NAME: string (nullable = false)
 |-- CROSS STREET NAME: string (nullable = false)
 |-- OFF STREET NAME: string (nullable = false)
 |-- NUMBER OF PERSONS INJURED: integer (nullable = true)
 |-- NUMBER OF PERSONS KILLED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: integer (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: integer (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: integer (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: integer (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: integer (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: integer (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = false)
 |-- CONTRIBUTING FACTOR VEHICLE 

In [178]:
# Check if format is correct
col.select("CRASH DATE").show(10)

+----------+
|CRASH DATE|
+----------+
|2021-04-13|
|2021-04-13|
|2021-04-12|
|2021-04-13|
|2021-04-11|
|2021-04-15|
|2019-05-21|
|2021-02-26|
|2021-03-09|
|2021-03-31|
+----------+
only showing top 10 rows



In [179]:
## Converting CRASH TIME type: String to timestamp 
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_timestamp

col= col.withColumn('CRASH TIME',to_timestamp(col["CRASH TIME"], 'H:mm'))
col.filter(col["CRASH TIME"].isNull()).show()

+----------+----------+-------+--------+--------+---------+--------+--------------+-----------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|BOROUGH|ZIP CODE|LATITUDE|LONGITUDE|LOCATION|ON STREET NAME|CROSS STREET NAME|OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VEHICLE 1|CONTRIBUTING FACTOR VEHICLE 2|CONTRIBUTING F

In [180]:
## Checking if format is correct
col.select("CRASH TIME").show(10)

+-------------------+
|         CRASH TIME|
+-------------------+
|1970-01-01 21:35:00|
|1970-01-01 16:00:00|
|1970-01-01 08:25:00|
|1970-01-01 17:30:00|
|1970-01-01 21:06:00|
|1970-01-01 20:00:00|
|1970-01-01 22:50:00|
|1970-01-01 14:50:00|
|1970-01-01 11:00:00|
|1970-01-01 22:20:00|
+-------------------+
only showing top 10 rows



In [181]:
### Removing YYYY-MM-DD from CRASH TIME
from pyspark.sql.types import DateType
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f

col = col.withColumn("CRASH TIME", f.date_format("CRASH TIME", 'HH:mm:ss'))


In [182]:
## Checking Results of Format
col.select("CRASH TIME").show(10)

+----------+
|CRASH TIME|
+----------+
|  21:35:00|
|  16:00:00|
|  08:25:00|
|  17:30:00|
|  21:06:00|
|  20:00:00|
|  22:50:00|
|  14:50:00|
|  11:00:00|
|  22:20:00|
+----------+
only showing top 10 rows



In [183]:
col.show()

+----------+----------+-------------+--------+---------+----------+--------------------+--------------------+-----------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME|      BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUT

In [184]:
col.write.csv('mycsv.csv')

Person data cleaning


In [185]:
# zip,borough,location,long,lat, [contributing1,2,3,4], killed and injured [3-4],COLLISION_ID.

In [186]:
ls


col.csv      ngrok-stable-linux-amd64.zip    spark-3.1.1-bin-hadoop2.7/
longlat.csv  ngrok-stable-linux-amd64.zip.1  spark-3.1.1-bin-hadoop2.7.tgz
mycsv.csv/   sample_data/                    spark-3.1.1-bin-hadoop2.7.tgz.1


In [187]:
!wget -q https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv   #Loaded person data for vehicle collisions

In [188]:
mv rows.csv person.csv

In [189]:
ls

col.csv                         person.csv
longlat.csv                     sample_data/
mycsv.csv/                      spark-3.1.1-bin-hadoop2.7/
ngrok-stable-linux-amd64.zip    spark-3.1.1-bin-hadoop2.7.tgz
ngrok-stable-linux-amd64.zip.1  spark-3.1.1-bin-hadoop2.7.tgz.1


In [190]:
person = spark.read.option("header",True).csv("./person.csv",inferSchema=True) # Read person data.

In [191]:
person.createOrReplaceTempView("person");

In [192]:
person.count()

4275261

In [193]:
len(person.columns)

21

In [194]:
person.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: string (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- PERSON_ID: string (nullable = true)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = true)
 |-- EMOTIONAL_STATUS: string (nullable = true)
 |-- BODILY_INJURY: string (nullable = true)
 |-- POSITION_IN_VEHICLE: string (nullable = true)
 |-- SAFETY_EQUIPMENT: string (nullable = true)
 |-- PED_LOCATION: string (nullable = true)
 |-- PED_ACTION: string (nullable = true)
 |-- COMPLAINT: string (nullable = true)
 |-- PED_ROLE: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = true)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = true)
 |-- PERSON_SEX: string (nullable = true)



In [195]:
person.count() - person.distinct().count() 

0

In [196]:
person.filter(person.COLLISION_ID.isNull()).drop()

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: string, CRASH_TIME: string, PERSON_ID: string, PERSON_TYPE: string, PERSON_INJURY: string, VEHICLE_ID: int, PERSON_AGE: int, EJECTION: string, EMOTIONAL_STATUS: string, BODILY_INJURY: string, POSITION_IN_VEHICLE: string, SAFETY_EQUIPMENT: string, PED_LOCATION: string, PED_ACTION: string, COMPLAINT: string, PED_ROLE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string, PERSON_SEX: string]

In [197]:
person.filter(person.UNIQUE_ID.isNull()).drop() 

DataFrame[UNIQUE_ID: int, COLLISION_ID: int, CRASH_DATE: string, CRASH_TIME: string, PERSON_ID: string, PERSON_TYPE: string, PERSON_INJURY: string, VEHICLE_ID: int, PERSON_AGE: int, EJECTION: string, EMOTIONAL_STATUS: string, BODILY_INJURY: string, POSITION_IN_VEHICLE: string, SAFETY_EQUIPMENT: string, PED_LOCATION: string, PED_ACTION: string, COMPLAINT: string, PED_ROLE: string, CONTRIBUTING_FACTOR_1: string, CONTRIBUTING_FACTOR_2: string, PERSON_SEX: string]

In [198]:
person.filter(person.PERSON_ID.isNull()).count()

19

In [199]:
person=person.fillna(value='Unknown',subset=["PERSON_ID"])

In [200]:
print(person.select("PERSON_TYPE").distinct().show(100))

+-----------+
|PERSON_TYPE|
+-----------+
|   Occupant|
| Pedestrian|
|  Bicyclist|
+-----------+

None


In [201]:
print(person.select("PERSON_INJURY").distinct().show(100)) #Already Clean

+-------------+
|PERSON_INJURY|
+-------------+
|       Killed|
|  Unspecified|
|      Injured|
+-------------+

None


In [202]:
person.select("VEHICLE_ID").distinct().count()

1957517

In [203]:
person.filter(person.VEHICLE_ID.isNull()).count()

167360

In [204]:

person=person.fillna(value='-999',subset=["VEHICLE_ID"])

In [205]:
person.filter(person.PERSON_AGE.isNull()).count()

371193

In [206]:

mean = spark.sql("Select sum(PERSON_AGE)/count(PERSON_AGE) from person where PERSON_AGE > 0 and PERSON_AGE < 100");

In [207]:
mean.show()

+-------------------------------------------------------------------------------------+
|(CAST(sum(CAST(PERSON_AGE AS BIGINT)) AS DOUBLE) / CAST(count(PERSON_AGE) AS DOUBLE))|
+-------------------------------------------------------------------------------------+
|                                                                    40.23597728618907|
+-------------------------------------------------------------------------------------+



In [208]:
person = person.withColumn('PERSON_AGE', when(person["PERSON_AGE"] > 100, -1).otherwise(person["PERSON_AGE"]))

In [209]:
person = person.withColumn('PERSON_AGE', when(person["PERSON_AGE"] < 0, -1).otherwise(person["PERSON_AGE"]))

In [210]:
person=person.fillna(value=-1,subset=["PERSON_AGE"])

In [211]:
person.select("PERSON_AGE").distinct().show()

+----------+
|PERSON_AGE|
+----------+
|        31|
|        85|
|        65|
|        53|
|        78|
|        34|
|        -1|
|        81|
|        28|
|        76|
|        27|
|        26|
|        44|
|        12|
|        91|
|        22|
|        93|
|        47|
|         1|
|        52|
+----------+
only showing top 20 rows



In [212]:
person.select("EJECTION").distinct().show()

+-----------------+
|         EJECTION|
+-----------------+
|Partially Ejected|
|   Does Not Apply|
|             null|
|          Unknown|
|      Not Ejected|
|          Ejected|
|          Trapped|
+-----------------+



In [213]:
person=person.fillna(value='Unknown',subset=["EJECTION"])  #Cleaned EJECTED

In [214]:
person = person.withColumn('EJECTION', when(person["EJECTION"] == "Unspecified", "Unknown").otherwise(person["EJECTION"]))

In [215]:
person.select("EMOTIONAL_STATUS").distinct().show()

+----------------+
|EMOTIONAL_STATUS|
+----------------+
|      Incoherent|
|  Does Not Apply|
|   Semiconscious|
|            null|
|         Unknown|
|     Unconscious|
|           Shock|
|  Apparent Death|
|       Conscious|
+----------------+



In [216]:
person=person.fillna(value='Unknown',subset=["EMOTIONAL_STATUS"])  #Cleaned EMOTIONAL_STATUS

In [217]:
person.select("BODILY_INJURY").distinct().show()

+--------------------+
|       BODILY_INJURY|
+--------------------+
|      Does Not Apply|
|                null|
|                Head|
|             Unknown|
|                Neck|
|Elbow-Lower-Arm-Hand|
|         Entire Body|
|                Face|
|                Back|
|       Hip-Upper Leg|
|    Abdomen - Pelvis|
|                 Eye|
|Shoulder - Upper Arm|
|               Chest|
| Knee-Lower Leg Foot|
+--------------------+



In [218]:
person=person.fillna(value='Unknown',subset=["BODILY_INJURY"])

In [219]:
person = person.withColumn('BODILY_INJURY', when(person["BODILY_INJURY"] == "Does N", "Unknown").otherwise(person["BODILY_INJURY"]))

In [220]:
person.select("PERSON_SEX").distinct().show()

+----------+
|PERSON_SEX|
+----------+
|         F|
|      null|
|         M|
|         U|
+----------+



In [221]:
person=person.fillna(value='Unspecified',subset=["PERSON_SEX"])  #Cleaned PERSON_SEX

In [222]:
person.select("POSITION_IN_VEHICLE").distinct().show()

+--------------------+
| POSITION_IN_VEHICLE|
+--------------------+
|Riding/Hanging on...|
|Front passenger, ...|
|Any person in the...|
|If one person is ...|
|      Does Not Apply|
|              Driver|
|Right rear passen...|
|                null|
|Middle front seat...|
|             Unknown|
|Left rear passeng...|
|Middle rear seat,...|
+--------------------+



In [223]:
person=person.fillna(value='Unknown',subset=["POSITION_IN_VEHICLE"])

In [224]:
person.select("SAFETY_EQUIPMENT").distinct().show(truncate = False)

+----------------------------------------+
|SAFETY_EQUIPMENT                        |
+----------------------------------------+
|None                                    |
|Stoppers Only (In-Line Skater/Bicyclist)|
|Pads Only (In-Line Skater/Bicyclist)    |
|null                                    |
|Air Bag Deployed                        |
|Air Bag Deployed/Lap Belt/Harness       |
|Air Bag Deployed/Child Restraint        |
|Unknown                                 |
|Other                                   |
|-                                       |
|Harness                                 |
|Helmet/Other (In-Line Skater/Bicyclist) |
|Child Restraint Only                    |
|Lap Belt/Harness                        |
|Helmet (Motorcycle Only)                |
|Lap Belt & Harness                      |
|Helmet Only (In-Line Skater/Bicyclist)  |
|Air Bag Deployed/Lap Belt               |
|Lap Belt                                |
+----------------------------------------+



In [225]:
person=person.fillna(value='Unknown',subset=["SAFETY_EQUIPMENT"])

In [226]:
person = person.withColumn('SAFETY_EQUIPMENT', when(person["SAFETY_EQUIPMENT"] == "-", "None").otherwise(person["SAFETY_EQUIPMENT"]))

In [227]:
person = person.withColumn('SAFETY_EQUIPMENT', when(person["SAFETY_EQUIPMENT"] == "Lap Belt/Harness", "Lap Belt & Harness").otherwise(person["SAFETY_EQUIPMENT"]))

In [228]:
person.select("PED_LOCATION").distinct().show(truncate = False)

+---------------------------------------------------------+
|PED_LOCATION                                             |
+---------------------------------------------------------+
|Does Not Apply                                           |
|null                                                     |
|Unknown                                                  |
|Pedestrian/Bicyclist/Other Pedestrian Not at Intersection|
|Pedestrian/Bicyclist/Other Pedestrian at Intersection    |
+---------------------------------------------------------+



In [229]:

person=person.fillna(value='Unknown',subset=["PED_LOCATION"])

In [230]:
person.select("PED_ACTION").distinct().show(truncate = False)


+-----------------------------------------------+
|PED_ACTION                                     |
+-----------------------------------------------+
|Pushing/Working on Car                         |
|Not in Roadway                                 |
|Riding/Walking Along Highway Against Traffic   |
|Crossing, No Signal, or Crosswalk              |
|Does Not Apply                                 |
|null                                           |
|Other Actions in Roadway                       |
|Unknown                                        |
|Working in Roadway                             |
|Getting On/Off Vehicle Other Than School Bus   |
|Emerging from in Front of/Behind Parked Vehicle|
|Playing in Roadway                             |
|Crossing Against Signal                        |
|Crossing, No Signal, Marked Crosswalk          |
|Going to/From Stopped School Bus               |
|Crossing With Signal                           |
|Riding/Walking Along Highway With Traffic      |


In [231]:
person=person.fillna(value='Unknown',subset=["PED_ACTION"])

In [232]:
person.select("COMPLAINT").distinct().show(truncate = False)

+----------------------------------+
|COMPLAINT                         |
+----------------------------------+
|Complaint of Pain                 |
|Minor Burn                        |
|Contusion - Bruise                |
|Does Not Apply                    |
|Severe Burn                       |
|null                              |
|Minor Bleeding                    |
|Crush Injuries                    |
|Paralysis                         |
|Unknown                           |
|Abrasion                          |
|Fracture - Dislocation            |
|None Visible                      |
|Internal                          |
|Whiplash                          |
|Fracture - Distorted - Dislocation|
|Concussion                        |
|Severe Bleeding                   |
|Severe Lacerations                |
|Complaint of Pain or Nausea       |
+----------------------------------+
only showing top 20 rows



In [233]:
person=person.fillna(value='Unknown',subset=["COMPLAINT"])

In [234]:
person.select("PED_ROLE").distinct().show()

+---------------+
|       PED_ROLE|
+---------------+
|        Witness|
|         Driver|
|           null|
|Notified Person|
|  Policy Holder|
|          Other|
|      Passenger|
|     Registrant|
|     Pedestrian|
|          Owner|
| In-Line Skater|
+---------------+



In [235]:
person=person.fillna(value='Unknown',subset=["PED_ROLE"])

In [236]:
person.select("CONTRIBUTING_FACTOR_1").distinct().show(100, truncate = False)

+-----------------------------------------------------+
|CONTRIBUTING_FACTOR_1                                |
+-----------------------------------------------------+
|Following Too Closely                                |
|Traffic Control Disregarded                          |
|Driverless/Runaway Vehicle                           |
|Windshield Inadequate                                |
|Using On Board Navigation Device                     |
|Unsafe Speed                                         |
|Shoulders Defective/Improper                         |
|Tinted Windows                                       |
|Oversized Vehicle                                    |
|Passing or Lane Usage Improper                       |
|Lane Marking Improper/Inadequate                     |
|Other Lighting Defects                               |
|null                                                 |
|Aggressive Driving/Road Rage                         |
|Other Vehicular                                

In [237]:
person = person.withColumn('CONTRIBUTING_FACTOR_1', when(person["CONTRIBUTING_FACTOR_1"] == "Unspecified", "Unknown").otherwise(person["CONTRIBUTING_FACTOR_1"]))

In [238]:
person=person.fillna(value='Unknown',subset=["CONTRIBUTING_FACTOR_1"])

In [239]:
person.select("CONTRIBUTING_FACTOR_2").distinct().show(100, truncate = False)

+-----------------------------------------------------+
|CONTRIBUTING_FACTOR_2                                |
+-----------------------------------------------------+
|Following Too Closely                                |
|Traffic Control Disregarded                          |
|Driverless/Runaway Vehicle                           |
|Accelerator Defective                                |
|Unsafe Speed                                         |
|Oversized Vehicle                                    |
|Passing or Lane Usage Improper                       |
|Lane Marking Improper/Inadequate                     |
|null                                                 |
|Aggressive Driving/Road Rage                         |
|Other Vehicular                                      |
|Driver Inexperience                                  |
|Texting                                              |
|Illnes                                               |
|Drugs (illegal)                                

In [240]:
person=person.fillna(value='Unknown',subset=["CONTRIBUTING_FACTOR_2"])

In [241]:
person = person.withColumn('CONTRIBUTING_FACTOR_2', when(person["CONTRIBUTING_FACTOR_2"] == "Unspecified", "Unknown").otherwise(person["CONTRIBUTING_FACTOR_2"]))

In [242]:
person.select("PED_ROLE").distinct().show()

+---------------+
|       PED_ROLE|
+---------------+
|        Witness|
|         Driver|
|Notified Person|
|  Policy Holder|
|        Unknown|
|          Other|
|      Passenger|
|     Registrant|
|     Pedestrian|
|          Owner|
| In-Line Skater|
+---------------+



In [243]:
person=person.fillna(value='Unknown',subset=["PED_ROLE"])

In [244]:
#Change CRASH_DATE from String to date
from pyspark.sql.types import DataType
from pyspark.sql.functions import to_date

person = person.withColumn('CRASH_DATE', to_date(person["CRASH_DATE"], 'MM/dd/yyyy'))
person.printSchema()


root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- PERSON_ID: string (nullable = false)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = false)
 |-- EMOTIONAL_STATUS: string (nullable = false)
 |-- BODILY_INJURY: string (nullable = false)
 |-- POSITION_IN_VEHICLE: string (nullable = false)
 |-- SAFETY_EQUIPMENT: string (nullable = false)
 |-- PED_LOCATION: string (nullable = false)
 |-- PED_ACTION: string (nullable = false)
 |-- COMPLAINT: string (nullable = false)
 |-- PED_ROLE: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = false)
 |-- PERSON_SEX: string (nullable = false)



In [245]:
from pyspark.sql.functions import to_timestamp
person = person.withColumn('CRASH_TIME', to_timestamp(person["CRASH_TIME"], 'H:mm'))
person.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: timestamp (nullable = true)
 |-- PERSON_ID: string (nullable = false)
 |-- PERSON_TYPE: string (nullable = true)
 |-- PERSON_INJURY: string (nullable = true)
 |-- VEHICLE_ID: integer (nullable = true)
 |-- PERSON_AGE: integer (nullable = true)
 |-- EJECTION: string (nullable = false)
 |-- EMOTIONAL_STATUS: string (nullable = false)
 |-- BODILY_INJURY: string (nullable = false)
 |-- POSITION_IN_VEHICLE: string (nullable = false)
 |-- SAFETY_EQUIPMENT: string (nullable = false)
 |-- PED_LOCATION: string (nullable = false)
 |-- PED_ACTION: string (nullable = false)
 |-- COMPLAINT: string (nullable = false)
 |-- PED_ROLE: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_1: string (nullable = false)
 |-- CONTRIBUTING_FACTOR_2: string (nullable = false)
 |-- PERSON_SEX: string (nullable = false)



In [246]:
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f
person = person.withColumn('CRASH_TIME', f.date_format("CRASH_TIME", 'HH:mm'))

In [247]:
person.select("CRASH_TIME").show(10)

+----------+
|CRASH_TIME|
+----------+
|     09:43|
|     15:15|
|     17:55|
|     13:05|
|     11:16|
|     19:15|
|     16:45|
|     01:15|
|     13:04|
|     08:20|
+----------+
only showing top 10 rows



In [248]:
person.select("CRASH_DATE").show(10)

+----------+
|CRASH_DATE|
+----------+
|2019-10-26|
|2019-10-25|
|2019-10-26|
|2016-11-21|
|2019-10-25|
|2019-10-24|
|2019-10-26|
|2019-10-26|
|2019-10-26|
|2019-10-24|
+----------+
only showing top 10 rows



In [253]:
person.repartition(1).write.format('com.databricks.spark.csv').save("./clean_person.csv",header = 'true')

In [254]:
col.repartition(1).write.format('com.databricks.spark.csv').save("./collision_clean.csv",header = 'true')